## 빅데이터 실습

### 스타벅스매장 입지 분석

#### 데이터 수집

##### 데이터 전처리 마무리

##### 스타벅스 분석 시각화

In [ ]:
# 판다스
import pandas as pd
# 시각화 라이브러리
import folium
import json

In [ ]:
# 스타벅스 서울시 매장정보 로드
dfStarbucksSeoul = pd.read_excel('./data/starbucks_seoul_list.xlsx')

In [ ]:
# 주소를 ' '로 자른다음(배열) 1번쨰 인덱스 값 구하기
guName = []
for address in dfStarbucksSeoul['주소']:
    addr = address.split(' ')
    guName.append(addr[1])

dfStarbucksSeoul['구']= guName

In [ ]:
dfStarbucksSeoul.to_excel('./data/최종_서울_스타벅스매장list.xlsx',index=False)

In [ ]:
# 스타벅스 분석 데이터 만들기
dfSeoulGuList = pd.read_excel('./data/서울_위경도.xlsx')
dfSeoulGuList.tail()
dfSeoulGuLast = dfSeoulGuList[['do','city','latitude','longitude']]
dfSeoulGuLast.to_excel('./data/최종_서울_구리스트.xlsx',index=False)

In [ ]:
# 각 칼럼의 영어이름은 한글 이름으로 변경
dfSeoulGuLast.columns = ['시','구','위도','경도']
dfSeoulGuLast.tail()
dfSeoulGuLast.to_excel('./data/최종_서울_구리스트.xlsx',index=False)

In [ ]:
# 스타벅스 매장에서 구로 그룹화, 매장명수 카운팅(pivot_table)
starbucksSeoulCount = dfStarbucksSeoul.pivot_table(values='매장명',
                             index='구',aggfunc='count').rename(columns={'매장명':'매장수'})
starbucksSeoulCount.tail()

In [ ]:
# 서울구 리스트에 스타벅스구별 매장수를 병합
lastSbSeoulCount = pd.merge(left=dfSeoulGuLast,right=starbucksSeoulCount,how='left',on='구')

In [ ]:
#서울시 구별 인구통계로드
dfSeoulGuPop = pd.read_excel('./data/수정_서울_구별인구.xlsx')
dfSeoulGuPop.tail()

In [ ]:
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount,right=dfSeoulGuPop[['구','성별소계']],how='left',on='구')
lastSbSeoulCount.tail()

In [ ]:
# 사업체 정보에 들어올 종사자수 컬럼이름이 '성별소계'로 동일, 컬럼이름 변경
lastSbSeoulCount.columns = ['시','구','위도','경도','스타벅스매장수','주민등록인구']

In [ ]:
# 사업체 정보 로드
dfSeoulGuBiz = pd.read_excel('./data/수정_서울_구별_사업체현황.xlsx')

In [ ]:
# 사업체정보에서 컬럼이름 부터 변경
dfSeoulGuBiz = dfSeoulGuBiz[['구','소계','성별소계']]
dfSeoulGuBiz.columns=['구','사업체수','종사가주']

In [ ]:
# 사업체정보를 스타벅스 최종데이터 병합
lastSbSeoulCount = pd.merge(left=lastSbSeoulCount,right=dfSeoulGuBiz,how='left',on='구')

In [ ]:
lastSbSeoulCount.to_excel('./data/최종_서울_스타벅스매장_데이터.xlsx',index=False)

### 시각화

In [ ]:
# 서울시 스타벅스 매장위치 데이터 로드
dfStarbucksSeoul = pd.read_excel('./data/최종_서울_스타벅스매장list.xlsx')

In [ ]:
# 데이터 불러오기
dfStarbucksSeoulCount = pd.read_excel('./data/최종_서울_스타벅스매장_데이터.xlsx')

In [ ]:
# 서울시 지도 생성
seoulLoc=[37.550800, 126.989857]
sbMap = folium.Map(location=seoulLoc,zoom_start=11)

# 각 매장 위치를 지도에 마커로 표시
for idx in dfStarbucksSeoul.index:
    lat = dfStarbucksSeoul.loc[idx,'위도']
    lng = dfStarbucksSeoul.loc[idx,'경도']
    storeType = dfStarbucksSeoul.loc[idx,'매장타입']
    storName = dfStarbucksSeoul.loc[idx,'매장명']

    # 매장타입별 색상
    fillColor = ''
    if storeType == 'general':
        fillColor = 'darkgreen'
        size = 2
    elif storeType == 'reserve':
        fillColor = 'black'
        size = 5
    elif storeType == 'generalDT':
        fillColor = 'orange'
        size = 3
    elif storeType == 'generalWT':
        fillColor = 'purple'
        size = 3        
    # 서클마커
    folium.CircleMarker(location=[lat,lng],popup=storName,
                        fill=True, fill_color=fillColor, fill_opacity=1,
                        color = 'yellow',weight=1,radius=size).add_to(sbMap)

sbMap

In [ ]:
# 서울시 구별 스타벅스 매장수 시각화
dfStarbucksSeoulCount.tail()

In [ ]:
# 서울시 행정구역(구별) 경계지도 파일
path = './data/seoul_municipalities_geo_simple.json'
seoulGeoSimple = json.load(open(path, encoding = 'utf-8'))

In [ ]:
seoulGeoSimple['features'][0]['properties']

In [ ]:
def style_function(feature):
    return{
        'opacity' : 0.9,
        'weight' : 2,
        'color' : 'white',
        'fillOpacity' : 0,
        'dashArray' : '5,5'
    }

In [ ]:
## 지도생성
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11,tiles='CartoDB.dark_matter')
folium.GeoJson(seoulGeoSimple,style_function=style_function).add_to(sbCountMap)
sbCountMap

In [ ]:
# 스타벅스 평균 매장수 계산
sbMean = dfStarbucksSeoulCount['스타벅스매장수'].mean()
sbMean

In [ ]:
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11,tiles='CartoDB.dark_matter')

folium.GeoJson(seoulGeoSimple,style_function=style_function).add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx,'구']

    if count > sbMean :
        fillColor = 'red'
    else:
        fillColor = 'blue'

    folium.CircleMarker(location=[lat,lng], popup = guName + '  ' + str(count),
                        color = '#FFFF00', fill_color=fillColor, fill_opacity=0.7,
                        weighht=2,radius=count/2).add_to(sbCountMap)

sbCountMap

In [ ]:
from folium.features import DivIcon

In [ ]:
# 구별 스타벅스 매장수 시각화
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11)

folium.GeoJson(seoulGeoSimple,style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구','스타벅스매장수'],
                  legend_name='구별 스타벅스매장수',
                  fill_color='YlGn', fill_opacity=0.7, line_opacity=0.5, key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'스타벅스매장수']
    guName = dfStarbucksSeoulCount.loc[idx,'구']

    folium.Marker(location=[lat,lng],icon=DivIcon(
        icon_size=(150,60),
        icon_anchor =(75,25),
        html = f'''
                <div style ="font-size : 9pt; font-weight: bold, color : black; text-alignLcenter;">
                    {guName} / {count}
                </div>  
                '''
    )).add_to(sbCountMap)
sbCountMap

In [ ]:
# 서울 구별 인구수 시각화
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11)

folium.GeoJson(seoulGeoSimple,style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구','주민등록인구'],
                  legend_name='구별 인구분포',
                  fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.5, key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'주민등록인구']
    guName = dfStarbucksSeoulCount.loc[idx,'구']

    folium.Marker(location=[lat,lng],icon=DivIcon(
        icon_size=(150,60),
        icon_anchor =(75,25),
        html = f'''
                <div style ="font-size : 9pt; font-weight: bold, color : black; text-alignLcenter;">
                    {guName} / {count}
                </div>  
                '''
    )).add_to(sbCountMap)
sbCountMap

In [ ]:
sbCountMap = folium.Map(location=seoulLoc,zoom_start=11)

folium.GeoJson(seoulGeoSimple,style_function=style_function).add_to(sbCountMap)

folium.Choropleth(geo_data=seoulGeoSimple, data=dfStarbucksSeoulCount, columns=['구','사업체수'],
                  legend_name='구별 사업체분포',
                  fill_color='YlGn', fill_opacity=0.7, line_opacity=0.5, key_on='properties.name').add_to(sbCountMap)

for idx in dfStarbucksSeoulCount.index:
    lat = dfStarbucksSeoulCount.loc[idx,'위도']
    lng = dfStarbucksSeoulCount.loc[idx,'경도']
    count = dfStarbucksSeoulCount.loc[idx,'사업체수']
    guName = dfStarbucksSeoulCount.loc[idx,'구']

    folium.Marker(location=[lat,lng],icon=DivIcon(
        icon_size=(150,60),
        icon_anchor =(75,25),
        html = f'''
                <div style ="font-size : 9pt; font-weight: bold, color : black; text-alignLcenter;">
                    {guName} / {count}
                </div>  
                '''
    )).add_to(sbCountMap)

sbCountMap

##### 결론 : 스타벅스는 인구수보다 직장인이 많이 분포한 구에 매장을 많이 오픈한다